In [1]:
import pickle
import pandas as pd
import os
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import wandb
from torch.utils.data.sampler import SubsetRandomSampler
import pprint
import matplotlib.pyplot as plt
import torch
with torch.profiler.profile() as profiler:
        pass
from sklearn.preprocessing import MinMaxScaler

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
def build_network(f1_layer_size, f2_layer_size, dropout, num_layers=2):


    network = nn.Sequential(  # fully-connected, dual hidden layer
        nn.Linear(128, f1_layer_size),
        nn.ReLU(),
        nn.Dropout(dropout),
        nn.Linear(f1_layer_size, f2_layer_size),
        nn.ReLU(),
        nn.Dropout(dropout),
        nn.Linear(f2_layer_size, 8),
        nn.Softmax(dim=1),
    )

    return network


In [4]:
rnn = nn.LSTMCell(10, 20)
input = torch.randn(2, 10)
h0 = torch.randn(2, 20)
c0 = torch.randn(2, 20)
output, (hn, cn) = rnn(input, (h0, c0))
hnt, cnt = hn,cn

In [5]:
print(f"{rnn=}\n,{input=},{h0=},{c0=}\n,{output=},{hn=}\n,{cn=}")

rnn=LSTMCell(10, 20)
,input=tensor([[ 0.4143, -2.2494,  0.4302, -0.6370, -0.2114,  0.8064, -1.6593, -0.3196,
          0.6626,  0.1947],
        [-0.1220, -1.1670, -0.1471, -0.6254, -0.4964, -0.9966,  0.1192,  1.0988,
          1.1638,  1.0839]]),h0=tensor([[-1.3160,  0.3458, -0.9221,  0.4392,  1.3843, -0.5240, -1.3394,  0.6273,
          2.1268, -0.3708, -0.8993, -1.0195, -0.7579, -0.2196, -0.6876, -0.6964,
         -0.6332, -1.7035, -1.1092, -2.5167],
        [ 0.7712, -0.7139, -0.3756,  0.6501,  1.6746,  0.2886, -2.3869, -0.3599,
          0.8924,  2.2568, -0.1269, -0.4855,  0.5185,  0.9146, -0.2467, -0.2268,
          1.3405, -0.0111,  1.3997, -1.3217]]),c0=tensor([[ 1.6252,  1.1010,  1.1989, -0.6024,  0.1357, -1.1412, -0.7005, -3.3551,
          0.1322, -1.1232,  0.2013,  0.5477,  0.6711,  0.7415,  0.5746, -0.2466,
          2.7686, -0.6592, -0.0422,  0.4848],
        [ 1.4166, -0.9735,  0.7672,  1.6384,  1.1774,  0.2098, -0.9295,  0.0250,
         -0.7137, -1.0848, -0.8239, -0.66

In [6]:
print(cnt.grad_fn.next_functions)
print(cn.grad_fn.next_functions)

((<AddBackward0 object at 0x0000028BD87BAEE0>, 0),)
((<AddBackward0 object at 0x0000028BD87BA8B0>, 0),)


In [43]:
class DogInput:
    def __init__(self, dogid, raceid,stats, dog) -> None:
        self.dogid= dogid
        self.raceid = raceid
        self.stats = stats
        self.dog = dog
        
    def lstm_i(self, lstmInput):
        self.lstmCell = lstmInput

    def nextrace(self, raceid):
        self.nextrace_id = raceid

    def lstm_o(self, lstm_o):
        self.dog.races[self.nextrace_id].lstm_i(lstm_o)


SyntaxError: invalid syntax (2967570734.py, line 8)

In [36]:
class Dog:
    def __init__(self, dogid, hidden_size, layers) -> None:
        self.dogid = dogid
        # self.raceids = raceids #possible dictionary of race id keys dog stat outs
        self.lstmcell = torch.randn(layers, hidden_size) 
        self.races = {}

    def add_races(self, raceid, racedate, stats,nextraceid):
        self.races[raceid] = DogInput(self.dogid, raceid, stats, self)
        self.races[raceid].nextrace(nextraceid)

In [37]:
class Race:
    def __init__(self, raceid,trackOHE, dist):
        self.raceid = raceid
        self.race_dist = dist
        self.race_track = trackOHE
        self.dogs = {}

    def add_dog(self, dogid, dog):
        self.dogs[dogid] = dog

    def nn_input(self):
        input = torch.cat([x.stats for x in self.dogs.values()], dim = 0)
        full_input = torch.cat((self.race_dist,self.race_track, input), dim=0)
        return full_input

    def lstm_input(self):
        l_input = [x.lstmCell for x in self.dogs.values()]
        return l_input

    def list_dogs(self):
        dogs_l = [x for x in self.dogs.values()]
        return dogs_l

In [66]:
class Net(nn.Module):
    def __init__(self):
      super(Net, self).__init__()
      self.lstm1 = nn.LSTMCell(128,64)
      self.lstm2 = nn.LSTMCell(128,64)
      self.lstm3 = nn.LSTMCell(128,64)
      self.lstm4 = nn.LSTMCell(128,64)
      self.lstm5 = nn.LSTMCell(128,64)
      self.lstm6 = nn.LSTMCell(128,64)
      self.lstm7 = nn.LSTMCell(128,64)
      self.lstm8 = nn.LSTMCell(128,64)
      self.fc2 = nn.Linear(128, 10)

    # x represents our data
    def forward(self, race:Race):
      x = race.nn_input()
      lstms = race.lstm_input()
      [n1,n2,n3,n4,n5,n6,n7,n8] = lstms

      # Obtain data from classes
      x1, (h1, c1) = self.lstm1(x, (x.dog1.LSTM))
      x2, (h2, c2) = self.lstm2(x, (x.dog2.LSTM))
      x3, (h3, c3) = self.lstm3(x, (x.dog3.LSTM))
      x4, (h4, c4) = self.lstm4(x, (x.dog4.LSTM))

      # Pass data through conv1
      xn1,(hn1,cn1) = self.lstm1(x, (h1, c1))
      xn2,(hn2,cn2) = self.lstm1(x, (h2, c2))
      xn3,(hn3,cn3) = self.lstm1(x, (h3, c3))
      xn4,(hn4,cn4) = self.lstm1(x, (h4, c4))

      # Apply softmax to x
      output = F.log_softmax(x, dim=1)
      return output

SyntaxError: invalid syntax (344025084.py, line 17)

In [64]:
class Races:
    def __init__(self, hidden_size, layers) -> None:
        self.racesDict = {}
        self.dogsDict = {}
        self.raceIDs = []
        self.hidden_size = hidden_size
        self.layers = layers

    def add_race(self,raceid:str, trackOHE, dist):
        self.racesDict[raceid] = Race(raceid, trackOHE, dist)
        self.raceIDs.append(raceid)

    def add_dog(self,dogid):
        if dogid not in self.dogsDict.keys():
            self.dogsDict[dogid] = Dog(dogid, self.hidden_size, self.layers)
        else:
            self.dogsDict[dogid] = self.dogsDict[dogid]

    def get_race_input(self, idx):
        raceidx = self.raceIDs[idx]
        input = torch.cat([x.stats for x in self.racesDict[raceidx].dogs.values()], dim = 0)
        full_input = torch.cat((self.racesDict[raceidx].race_dist,self.racesDict[raceidx].race_track, input), dim=0 )
        dogs = [x for x in self.racesDict[raceidx].dogs.values()]
        return (full_input, dogs)


In [39]:
dog = DogInput(1233,1000, 1)
dog.stats

dog.lstm(1)
dog.lstmCell

TypeError: __init__() missing 1 required positional argument: 'dog'

In [ ]:
os.getcwd()
os.chdir(r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA")
dog_stats_file = open( 'dog_stats_df.npy', 'rb')
resultsdf = pickle.load(dog_stats_file)
dog_stats_df = pd.DataFrame(resultsdf)

In [46]:
dog_stats_df[dog_stats_df['dogid']=="-159632"]

,dogid,raceid,date,trackOHE,dist,stats,place,startprice,margin,bfSP
2524,-159632,132951399,2016-07-11,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",450.0,"[6, 0.059524999999999995, nan, 0.0595249999999...",7.0,17.0,9.00,25.000000
8743,-159632,139325114,2016-08-02,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",400.0,"[8, 0.05930694444444444, 0.0003083771240174653...",6.0,11.2,6.70,18.766242
20697,-159632,151780470,2016-09-13,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",460.0,"[1, 0.05887962962962962, 0.0007715841355220415...",5.0,19.3,5.96,16.990319
44437,-159632,177083976,2016-12-05,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",450.0,"[3, 0.058605374396135265, 0.000835319370773274...",8.0,19.0,10.10,37.180216
71516,-159632,1967920009,2016-01-26,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",400.0,"[8, nan, nan, nan, nan, nan, nan, nan, nan, na...",5.0,3.8,14.33,NaN


In [40]:
dog_stats_group = dog_stats_df.groupby(["dogid"])
raceDB = Races(64,1)
print(raceDB)
for i,j in tqdm(dog_stats_group):
    #print(f"dog id = {i}\nlist = {j}")
    #tqdm.write(f"{i}")

    #creates next race column
    j["next_race"] = j["raceid"].shift(-1)
    raceDB.add_dog(i)
    
    j.apply(lambda x: raceDB.dogsDict[i].add_races(x['raceid'], x['date'], torch.Tensor(x['stats']),x['next_race']), axis=1)
    



  0%|          | 0/25187 [00:00<?, ?it/s]

100%|██████████| 25187/25187 [00:45<00:00, 551.47it/s]


In [41]:
races_group = dog_stats_df.groupby(['raceid'])

for i,j in tqdm(races_group):
    dist = torch.tensor([j.dist.iloc[0]])
    trackOHE = torch.tensor(j.trackOHE.iloc[0])
    raceDB.add_race(i,trackOHE,dist)
    j.apply(lambda x: raceDB.racesDict[i].add_dog(x['dogid'], raceDB.dogsDict[x['dogid']].races[i]), axis=1)

100%|██████████| 72073/72073 [00:57<00:00, 1243.15it/s]


In [63]:
raceDB.dogsDict["-159632"].races["132951399"].dog

In [ ]:
x = raceDB.get_race_input(5)
x.shape

AttributeError: 'DogInput' object has no attribute 'lstmCell'

In [67]:
[l1,l2,l3,l4,l5,l6,l7,l8] = [1,2,3,4,5,6,7,8]